Adapted from https://github.com/catalystneuro/movshon-lab-to-nwb/blob/main/tutorials/blackrock_nwb_conversion_detailed.ipynb

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [120]:
import os
import re
import datetime

dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw_5s'
filename = os.path.join(dir_name, base_filename)
print(filename)

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s


In [3]:
!ls ../nwb-conversion-tools/nwb_conversion_tools -l

total 76
-rwxrwxrwx 1 steburg steburg    69 Apr  5 11:28 __init__.py
-rwxrwxrwx 1 steburg steburg  1404 Apr  5 11:28 auto_qc.py
-rwxrwxrwx 1 steburg steburg  1338 Apr  5 11:28 basedatainterface.py
-rwxrwxrwx 1 steburg steburg  2134 Apr  5 11:28 baseimagingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2775 Apr  5 11:28 baselfpextractorinterface.py
-rwxrwxrwx 1 steburg steburg  4629 Apr  5 11:28 baserecordingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2363 Apr  5 11:28 basesegmentationextractorinterface.py
-rwxrwxrwx 1 steburg steburg  3829 Apr  5 11:28 basesortingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  4312 Apr  5 11:28 conversion_tools.py
drwxrwxrwx 1 steburg steburg  4096 Apr  5 11:28 datainterfaces
-rwxrwxrwx 1 steburg steburg  5100 Apr  5 11:28 json_schema_utils.py
-rwxrwxrwx 1 steburg steburg 10154 Apr  5 11:28 metafile.schema.json
-rwxrwxrwx 1 steburg steburg  6665 Apr  5 11:28 nwbconverter.py
drwxrwxrwx 1 steburg steburg  4096 Apr  5 11:28 schemas
-rwxrw

In [4]:
# Import modules

import random
import string
import pytz
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile
import numpy as np

from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface,  BaseDataInterface
from nwb_conversion_tools.basesortingextractorinterface import BaseSortingExtractorInterface
from nwb_conversion_tools.json_schema_utils import get_schema_from_method_signature, get_base_schema, fill_defaults
from nwb_conversion_tools.datainterfaces.interface_utils.brpylib import NsxFile
from nwb_conversion_tools import SpikeGLXRecordingInterface

In [5]:
# from basedatainterface.py
metadata_schema = get_base_schema(
    id_='metadata.schema.json',
    root=True,
    title='Metadata',
    description='Schema for the metadata',
    version="0.1.0",
)

In [6]:
metadata_schema

{'required': [],
 'properties': {},
 'type': 'object',
 'additionalProperties': False,
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'metadata.schema.json',
 'title': 'Metadata',
 'description': 'Schema for the metadata',
 'version': '0.1.0'}

In [7]:
glx = SpikeGLXRecordingInterface

In [8]:
glx.get_source_schema()

{'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to SpikeGLX file.'}},
 'type': 'object',
 'additionalProperties': True}

In [9]:
import inspect

inspect.signature(glx.__init__).parameters

mappingproxy({'self': <Parameter "self">,
              'file_path': <Parameter "file_path: Union[str, pathlib.Path, NoneType]">,
              'stub_test': <Parameter "stub_test: Union[bool, NoneType] = False">})

In [12]:
test = dict(
            Ecephys=dict(
                Device=[
                    dict(
                        description="Device for the NWB Conversion Tools tutorial."
                    )
                ],
                ElectrodeGroup=[
                    dict(
                        name="ElectrodeGroup",
                        description="Electrode group for the NWB Conversion Tools tutorial."
                    )
                ],
                Electrodes=[
                    dict(
                        name="group_name",
                        description="Custom ElectrodeGroup name for these electrodes.",
                        data=["ElectrodeGroup" for x in range(1)]
                    ),
                    dict(
                        name="custom_electrodes_column",
                        description="Custom column in the electrodes table for the NWB Conversion Tools tutorial.",
                        data=[x for x in range(1)]
                    )
                ],
                ElectricalSeries=dict(
                    name="ElectricalSeries",
                    description="Raw acquisition traces for the NWB Conversion Tools tutorial."
                )
            )
        )

In [14]:
test = {'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to SpikeGLX file.'}},
 'type': 'object',
 'additionalProperties': True}

In [15]:
test

{'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to SpikeGLX file.'}},
 'type': 'object',
 'additionalProperties': True}

In [42]:
!ls ../spikeextractors/spikeextractors -l

total 224
-rwxrwxrwx 1 steburg steburg    86 Mar 24 10:00 MANIFEST.in
-rwxrwxrwx 1 steburg steburg  1265 Mar 19 08:48 __init__.py
drwxrwxrwx 1 steburg steburg  4096 Apr  5 09:53 __pycache__
-rwxrwxrwx 1 steburg steburg 22834 Mar 19 08:52 baseextractor.py
-rwxrwxrwx 1 steburg steburg  9269 Mar 19 08:52 cacheextractors.py
drwxrwxrwx 1 steburg steburg  4096 Mar 24 10:00 example_datasets
-rwxrwxrwx 1 steburg steburg   105 Mar 19 08:48 exceptions.py
-rwxrwxrwx 1 steburg steburg 36890 Mar 19 08:52 extraction_tools.py
-rwxrwxrwx 1 steburg steburg  5952 Apr  1 17:10 extractorlist.py
drwxrwxrwx 1 steburg steburg  4096 Mar 22 09:47 extractors
-rwxrwxrwx 1 steburg steburg  6494 Mar 19 08:52 multirecordingchannelextractor.py
-rwxrwxrwx 1 steburg steburg  8390 Mar 19 08:52 multirecordingtimeextractor.py
-rwxrwxrwx 1 steburg steburg  5458 Mar 19 08:52 multisortingextractor.py
-rwxrwxrwx 1 steburg steburg 41357 Mar 19 08:52 recordingextractor.py
-rwxrwxrwx 1 steburg steburg 30642 Mar 19 08:52 sorting

In [45]:
from spikeextractors.extractors.neoextractors import AxonaRecordingExtractor

In [53]:
re = AxonaRecordingExtractor(filename=filename)
print('Number of channels:', re.get_num_channels())
print('Channel groups:', re.get_channel_groups())

Number of channels: 16
Channel groups: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


Parse .set file for metadata to include in metadata.

In [191]:
def parse_generic_header(filename, params):
    """
    Given a binary file with phrases and line breaks, enters the
    first word of a phrase as dictionary key and the following
    string (without linebreaks) as value. Returns the dictionary.
    
    INPUT
    filename (str): .set file path and name.
    params (list or set): parameter names to search for. 
    
    OUTPUT
    header (dict): dictionary with keys being the parameters that
                   were found & values being strings of the data.
                   
    EXAMPLE
    parse_generic_header('myset_file.set', ['experimenter', 'trial_time'])
    """
    header = {}
    params = set(params)
    with open(filename, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                break
            line = bin_line.decode('cp1252').replace('\r\n', '').replace('\r', '').strip()
            parts = line.split(' ')
            key = parts[0]
            if key in params:
                header[key] = ' '.join(parts[1:])
            
    return header

In [192]:
set_file = filename+'.set'

In [193]:
parse_generic_header(set_file, params_of_interest)

{'trial_date': 'Sunday, 4 Oct 2020',
 'trial_time': '11:07:07',
 'experimenter': 'Abid',
 'comments': '',
 'duration': '600.00625',
 'sw_version': '1.2.2.16',
 'tracker_version': '0',
 'stim_version': '1',
 'audio_version': '0',
 'pretrigSamps': '10',
 'spikeLockout': '40'}

In [194]:
def read_iso_datetime(set_file):
    """ 
    Creates datetime object (y, m, d, h, m, s) from .set file header 
    """
    with open(set_file, 'r', encoding='cp1252') as f:
        for line in f:
            if line.startswith('trial_date'):
                date_string = re.findall(r'\d+\s\w+\s\d{4}$', line)[0]
            if line.startswith('trial_time'):
                time_string = line[len('trial_time')+1::].replace('\n', '')

    return datetime.datetime.strptime(date_string + ', ' + time_string, \
        "%d %b %Y, %H:%M:%S").isoformat()

In [183]:
class AxonaRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor."""

    RX = se.AxonaRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        source_schema = {
            'required': ['file_path'],
            'properties': {
                'file_path': {
                    'type': 'string',
                    'format': 'file',
                    'description': 'Path to Axona files.'
                }
            },
            'type': 'object',
            'additionalProperties': True
        }
        return source_schema
       
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def get_metadata(self):
        """Auto-fill as much of the metadata as possible. Must comply with metadata schema."""        
                
        # Extract information for specific parameters from .set file
        params_of_interest = [
            'experimenter', 
            'comments',
            'duration', 
            'sw_version',
            'tracker_version',
            'stim_version',
            'audio_version'
        ]
        set_file = self.source_data['filename']+'.set'
        par = parse_generic_header(set_file, params_of_interest)
        
        # Extract information from AxonaRecordingExtractor
        elec_group_names = RX.get_channel_groups()
        unique_elec_group_names = set(elec_group_names)
        
        # Add available metadata
        # TODO what else to add for cont. data? Are these always available? 
        # Create defaults for when sth is not available
        metadata = super().get_metadata()
        metadata['NWBFile'] = dict(
            session_start_time=read_iso_datetime(set_file),
            session_description=par['comments'],
            identifier=''.join(random.choices(string.ascii_uppercase + string.digits, k=12)),
            session_duration=par['duration']+'s',
            experimenter=par['experimenter'],
            tracker_version=par['tracker_version'],
            stim_version=par['stim_version'],
            audio_version=par['audio_version']
        )
        
        metadata['Ecephys'] = dict(
            Ecephys=dict(
                Device=[
                    dict(
                        name="Axona",
                        description="Axona DacqUSB, sw_version={}".format(header['sw_version']),
                        manufacturer="Axona"
                    )
                ],
                ElectrodeGroup=[
                    dict(
                        name=f'Group{group_name}',
                        description=f"Group {group_name} electrodes.",
                    )
                    for group_name in unique_elec_group_names
                ],
                Electrodes=[
                    dict(
                        name='group_name',
                        description="The name of the ElectrodeGroup this electrode is a part of.",
                        data=[f"Group{x}" for x in elec_group_names]
                    )
                ],
                ElectricalSeries=dict(
                    name='ElectricalSeries',
                    description="Raw acquisition traces."
                )
            )
        )
  
        return metadata

In [189]:
RXI = AxonaRecordingExtractorInterface(filename=filename)

In [190]:
RXI.get_metadata()

{'NWBFile': {'session_start_time': '2020-10-04T11:07:07',
  'session_description': '',
  'identifier': 'RAU7Q8AALO4M',
  'session_duration': '600.00625s',
  'experimenter': 'Abid'},
 'Ecephys': {'Ecephys': {'Device': [{'name': 'Axona',
     'description': 'Axona DacqUSB, sw_version=1.2.2.16',
     'manufacturer': 'Axona'}],
   'ElectrodeGroup': [{'name': 'Group0',
     'description': 'Group 0 electrodes.'}],
   'Electrodes': [{'name': 'group_name',
     'description': 'The name of the ElectrodeGroup this electrode is a part of.',
     'data': ['Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0',
      'Group0']}],
   'ElectricalSeries': {'name': 'ElectricalSeries',
    'description': 'Raw acquisition traces.'}}}}

In [186]:
base_metadata_schema = RXI.get_metadata_schema()

In [187]:
import json
print(json.dumps(base_metadata_schema, indent=2))

{
  "required": [],
  "properties": {
    "Ecephys": {
      "required": [
        "Device",
        "ElectrodeGroup",
        "ElectricalSeries"
      ],
      "properties": {
        "Device": {
          "required": [
            "name"
          ],
          "properties": {
            "name": {
              "description": "the name of this device",
              "type": "string"
            },
            "description": {
              "description": "Description of the device (e.g., model, firmware version, processing software version, etc.)",
              "type": "string"
            },
            "manufacturer": {
              "description": "the name of the manufacturer of this device",
              "type": "string"
            }
          },
          "type": "object",
          "additionalProperties": false,
          "tag": "pynwb.device.Device"
        },
        "ElectrodeGroup": {
          "required": [
            "name",
            "description",
            "lo

# Movshon lab - Blackrock Converter
This tutorial follows the step-by-step guide for a [NWB Converter](https://github.com/catalystneuro/nwb-conversion-tools/blob/master/documentation/conversion_tools_structure.md#step-by-step-operations)

In [3]:
from hussaini_lab_to_nwb import HussainiNWBConverter
from pynwb import NWBFile, NWBHDF5IO
from nwbwidgets import nwb2widget
from pathlib import Path
import yaml
import pprint

## Step 1 - Converter.get_source_schema()

In [6]:
# Get source_schema
source_schema = HussainiNWBConverter.get_source_schema()
pprint.pprint(source_schema['properties'], width=120)

{'BlackrockRecordingExtractorInterface': {'additionalProperties': True,
                                          'properties': {'filename': {'description': 'Path to Blackrock file.',
                                                                      'format': 'file',
                                                                      'type': 'string'},
                                                         'nsx_to_load': {'type': 'number'}},
                                          'required': ['filename'],
                                          'type': 'object'},
 'BlackrockSortingExtractorInterface': {'additionalProperties': True,
                                        'properties': {'filename': {'description': 'Path to Blackrock file.',
                                                                    'format': 'file',
                                                                    'type': 'string'}},
                                        'required': ['filename'

## Step 2 - Get user-input source_data that complies to the returned full source_schema

In [4]:
# Source data
base_path = Path('/media/luiz/storage/taufferconsulting/client_ben/project_movshon/data/')
file_recording = str(base_path / 'XX_LE_textures_20191128_002.ns3')
file_sorting = str(base_path / 'XX_LE_textures_20191128_002.nev')

source_data = dict(
    BlackrockRecordingExtractorInterface=dict(
        filename=file_recording,
        nsx_to_load=3
    ),
    BlackrockSortingExtractorInterface=dict(
        filename=file_sorting,
        nsx_to_load=6
    )
)

pprint.pprint(source_data, width=120)

{'BlackrockRecordingExtractorInterface': {'filename': '/media/luiz/storage/taufferconsulting/client_ben/project_movshon/data/XX_LE_textures_20191128_002.ns3',
                                          'nsx_to_load': 3},
 'BlackrockSortingExtractorInterface': {'filename': '/media/luiz/storage/taufferconsulting/client_ben/project_movshon/data/XX_LE_textures_20191128_002.nev',
                                        'nsx_to_load': 6}}


## Step 3 - Instantiate Converter

In [79]:
# Initialize converter
converter = MovshonBlackrockNWBConverter(source_data=source_data)

print('Data interfaces for this converter:')
pprint.pprint(converter.data_interface_objects, width=120)

NameError: name 'MovshonBlackrockNWBConverter' is not defined

## Step 4 - Converter.get_metadata_schema()

In [78]:
# Get metadata_schema
metadata_schema = converter.get_metadata_schema()
pprint.pprint(metadata_schema, width=80)

NameError: name 'converter' is not defined

## Step 5 - Automatically fetches available metadata with Converter.get_metadata()

In [7]:
# Get metadata from source data
metadata = converter.get_metadata()
pprint.pprint(metadata, width=120)

{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Device_ecephys'}],
             'ElectrodeGroup': [],
             'LFPElectricalSeries': {}},
 'NWBFile': {'identifier': 'JES40X1FWSHT',
             'session_description': '',
             'session_start_time': datetime.datetime(2019, 11, 28, 22, 35, 54, 729000, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 6 - Get user-input metadata

In [8]:
metadata['NWBFile']['session_description'] = 'example conversion'
pprint.pprint(metadata, width=120)

{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Device_ecephys'}],
             'ElectrodeGroup': [],
             'LFPElectricalSeries': {}},
 'NWBFile': {'identifier': 'JES40X1FWSHT',
             'session_description': 'example conversion',
             'session_start_time': datetime.datetime(2019, 11, 28, 22, 35, 54, 729000, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 7 - Converter.get_conversion_options_schema()

In [9]:
# Not used for this converter

## Step 8 - Get user-input conversion options

In [10]:
# Not used for this converter

## Step 9 - Run conversion user filled metadata and conversion_options

In [11]:
output_file = 'out_example.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file, 
    save_to_file=True,
    conversion_options=None
)

/home/luiz/anaconda3/envs/env_klusta/lib/python3.7/site-packages/hdmf/common/table.py:415: UserWarning: An attribute 'name' already exists on DynamicTable 'electrodes' so this column cannot be accessed as an attribute, e.g., table.name; it can only be accessed using other methods, e.g., table['name'].
  warn(msg)


NWB file saved at out_example.nwb!


## Final 1 - Check NWB file

In [12]:
# load file
fname = 'out_example.nwb'
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

root pynwb.file.NWBFile at 0x140281502197008
Fields:
  devices: {
    Device <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  file_create_date: [datetime.datetime(2021, 3, 5, 18, 47, 48, 213380, tzinfo=tzoffset(None, 3600))]
  identifier: JES40X1FWSHT
  processing: {
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_description: example conversion
  session_start_time: 2019-11-28 22:35:54.729000-05:00
  timestamps_reference_time: 2019-11-28 22:35:54.729000-05:00
  units: units <class 'pynwb.misc.Units'>



## Final 2 - Check NWB file with widgets

In [13]:
io = NWBHDF5IO(fname, 'r')
nwbfile = io.read()
nwb2widget(nwbfile)